# Neural Network Weight Optimisation Using Genetic Algorithms

## Tensorflow and PyGAD

### Importing Libararies

In [1]:
import sklearn
import pandas as pd
import pygad
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from pygad.kerasga import KerasGA

2023-04-03 22:07:24.669202: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 22:07:25.410557: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Configuring GPU

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2023-04-03 22:07:26.363098: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-03 22:07:26.492630: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-03 22:07:26.492813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

### Loading and Preprocessing the Data

In [3]:
from sklearn.datasets import load_iris

# Load the iris dataset
iris = load_iris()

# Print the feature names and target names
print("Feature names:", iris.feature_names)
print("Target names:", iris.target_names)

Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target names: ['setosa' 'versicolor' 'virginica']


In [4]:
df = pd.DataFrame(iris.data,columns = iris.feature_names)

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
sc = StandardScaler()

In [7]:
df = sc.fit_transform(df)

In [8]:
df = pd.DataFrame(df,columns = iris.feature_names)

In [9]:
df['labels'] = iris.target

In [10]:
X = df.iloc[:,:4]
y = df.iloc[:,-1]

### Splitting it into train and test and converting target into categorical data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                    test_size=0.33, 
                                                    random_state=42,
                                                    shuffle = True)

In [12]:
y_train=np_utils.to_categorical(y_train,num_classes=3)
y_test=np_utils.to_categorical(y_test,num_classes=3)

### Defining a basic tensorflow ANN 

In [13]:
model = Sequential([tf.keras.layers.Dense(4,input_shape = (4,),activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(16, activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(3,activation = 'softmax')
                   ])

In [14]:
model.compile(optimizer = 'adam', metrics = ['accuracy'])

### Fitness function for the genetic algorithm

In [15]:
def fitness_func(solution, sol_idx):
    global X_train, y_train, keras_ga, model
    
    model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model = model,weights_vector = solution)
    
    model.set_weights(weights = model_weights_matrix)
    
    predictions = model.predict(X_train)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    solution_fitness = 1.0/(loss(y_train,predictions).numpy() + 0.0000000001)
    
    return solution_fitness

In [16]:
def callback_generation(ga_instance): #just prints the current generation number and the fitness value of the best solution in the current generation
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))

### Instantiating kerasGA

In [17]:
keras_ga = KerasGA(model = model, num_solutions = 35)

In [27]:
num_generations = 100
num_parents_mating = 5
initial_population = keras_ga.population_weights

ga_instance = pygad.GA(num_generations=num_generations, 
                       num_parents_mating=num_parents_mating, 
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       on_generation=callback_generation,
                       parent_selection_type = 'rws',
                      
                       crossover_probability = 1,
                       mutation_type = 'random',
                       #mutation_probability = 0.7,
                       random_mutation_min_val = -4,
                       random_mutation_max_val = 4,
                       #parallel_processing = ['thread', None],
                      random_seed = 42,
                      )

### Evolving through the generations

In [28]:
with tf.device('/device:GPU:0'):
     ga_instance.run()

4/4 [==============================] - 0s 1ms/step
Generation = 1
4/4 [==============================] - 0s 886us/step
Fitness    = 0.99052890140017
4/4 [==============================] - 0s 832us/step
Generation = 2
4/4 [==============================] - 0s 861us/step
Fitness    = 0.99052890140017
4/4 [==============================] - 0s 892us/step
Generation = 3
4/4 [==============================] - 0s 841us/step
Fitness    = 0.99052890140017
4/4 [==============================] - 0s 886us/step


Generation = 4
4/4 [==============================] - 0s 894us/step
Fitness    = 0.99052890140017
4/4 [==============================] - 0s 914us/step
Generation = 5
4/4 [==============================] - 0s 875us/step
Fitness    = 0.99052890140017
4/4 [==============================] - 0s 1ms/step
Generation = 6
4/4 [==============================] - 0s 1ms/step
Fitness    = 0.99052890140017
4/4 [==============================] - 0s 835us/step
Generation = 7
4/4 [==============================] - 0s 838us/step
Fitness    = 0.99052890140017
4/4 [==============================] - 0s 904us/step


Generation = 8
4/4 [==============================] - 0s 850us/step
Fitness    = 1.0218787781397118
4/4 [==============================] - 0s 907us/step
Generation = 9
4/4 [==============================] - 0s 1ms/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 1ms/step
Generation = 10
4/4 [==============================] - 0s 840us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 920us/step
Generation = 11
4/4 [==============================] - 0s 905us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 877us/step


Generation = 12
4/4 [==============================] - 0s 942us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 856us/step
Generation = 13
4/4 [==============================] - 0s 839us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 887us/step
Generation = 14
4/4 [==============================] - 0s 851us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 965us/step
Generation = 15
4/4 [==============================] - 0s 860us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 880us/step


4/4 [==============================] - 0s 831us/step
Generation = 16
4/4 [==============================] - 0s 848us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 850us/step
Generation = 17
4/4 [==============================] - 0s 876us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 858us/step
Generation = 18
4/4 [==============================] - 0s 905us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 967us/step
Generation = 19
4/4 [==============================] - 0s 885us/step
Fitness    = 1.2025536786316804
4/4 [==============================] - 0s 833us/step


...

4/4 [==============================] - 0s 837us/step
Generation = 84
4/4 [==============================] - 0s 1ms/step
Fitness    = 1.8407605200969375
4/4 [==============================] - 0s 881us/step
Generation = 85
4/4 [==============================] - 0s 916us/step
Fitness    = 1.8407605200969375
4/4 [==============================] - 0s 858us/step
Generation = 86
4/4 [==============================] - 0s 846us/step
Fitness    = 1.8407605200969375
4/4 [==============================] - 0s 878us/step
Generation = 87
4/4 [==============================] - 0s 812us/step
Fitness    = 1.8407605200969375
4/4 [==============================] - 0s 855us/step


4/4 [==============================] - 0s 893us/step
Generation = 88
4/4 [==============================] - 0s 953us/step
Fitness    = 1.8407605200969375
4/4 [==============================] - 0s 982us/step
Generation = 89
4/4 [==============================] - 0s 925us/step
Fitness    = 1.8407605200969375
4/4 [==============================] - 0s 854us/step
Generation = 90
4/4 [==============================] - 0s 855us/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 831us/step
Generation = 91
4/4 [==============================] - 0s 842us/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 839us/step


4/4 [==============================] - 0s 899us/step
Generation = 92
4/4 [==============================] - 0s 932us/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 881us/step
Generation = 93
4/4 [==============================] - 0s 876us/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 865us/step
Generation = 94
4/4 [==============================] - 0s 910us/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 856us/step
Generation = 95
4/4 [==============================] - 0s 878us/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 888us/step


4/4 [==============================] - 0s 845us/step
Generation = 96
4/4 [==============================] - 0s 1ms/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 978us/step
Generation = 97
4/4 [==============================] - 0s 902us/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 1ms/step
Generation = 98
4/4 [==============================] - 0s 1ms/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 948us/step
Generation = 99
4/4 [==============================] - 0s 861us/step
Fitness    = 1.8461860794390772
4/4 [==============================] - 0s 959us/step


4/4 [==============================] - 0s 943us/step
Generation = 100
4/4 [==============================] - 0s 898us/step
Fitness    = 2.260982814246111


In [29]:
# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

4/4 [==============================] - 0s 2ms/step
Fitness value of the best solution = 2.260982814246111
Index of the best solution : 9


In [30]:
# Fetch the parameters of the best solution.
best_solution_weights = pygad.kerasga.model_weights_as_matrix(model=model,
                                                              weights_vector=solution)
model.set_weights(best_solution_weights)
predictions = model.predict(X_train)
# print("Predictions : \n", predictions)

# Calculate the categorical crossentropy for the trained model.
cce = tf.keras.losses.CategoricalCrossentropy()
print("Categorical Crossentropy : ", cce(y_train, predictions).numpy())

# Calculate the classification accuracy for the trained model.
ca = tf.keras.metrics.CategoricalAccuracy()
ca.update_state(y_train, predictions)
accuracy = ca.result().numpy()
print("Accuracy : ", accuracy)

4/4 [==============================] - 0s 829us/step
Categorical Crossentropy :  0.44228554
Accuracy :  0.86


In [34]:
test_pred = model.predict(X_test)
ca.reset_states()
ca.update_state(y_test,test_pred)
accuracy = ca.result().numpy()
print("Testing Accuracy : ", accuracy)

2/2 [==============================] - 0s 1ms/step
Testing Accuracy :  0.82


### This model has the following parameters:

Number of Solutions = 35

Number of Parents = 5

Number of Generations = 100

It produces a model with a training accuracy of 86% and testing accuracy of 82%